# Scraping CareerJet!


https://www.careerjet.com.mt/

We first load the libraries.

In [ ]:
!apt install chromium-chromedriver
!pip install selenium
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

from selenium.webdriver.common.keys import Keys
import pandas as pd
from time import sleep
import requests
from bs4 import BeautifulSoup
import cv2
from google.colab.patches import cv2_imshow

from google.colab import drive
import os
drive.mount("/content/drive")
pathOut = "/content/drive/MyDrive/Colab Notebooks/Outputs"
os.chdir(pathOut)

import sqlite3
import datetime

# Adding data to the current database

In [ ]:
wd = webdriver.Chrome(options=options)

def CreateAppendDB(jobdata):
    db=sqlite3.connect("MaltaJobs.db")
    db.row_factory=sqlite3.Row
    db.execute("""create table if not exists 
    Jobs(url TEXT PRIMARY KEY        NOT NULL,
         title              TEXT     NOT NULL,
         company            TEXT     NOT NULL,
         date_added         TEXT     NOT NULL
         );""")
    for x in range(0,len(jobdata)):
      try:
        db.execute("INSERT into Jobs(url,title,company,date_added) values(?,?,?,?)",
                   (jobdata["url"][x],jobdata["title"][x],jobdata["company"][x],jobdata["date_added"][x]))
      except Exception as e:
        print(str(e))
    db.commit()

#@title
def scrolldown(N):
    for z in range(1,N):
        wd.find_element_by_css_selector("body").send_keys(Keys.ARROW_DOWN)

def ScrapePage(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    return soup

def SearchScrape(role,location):
    sleep(1)
    wd.get("https://www.careerjet.com.mt/")
    wd.find_element_by_id("s").send_keys(role)
    wd.find_element_by_id("l").send_keys(location)
    sleep(1)
    wd.find_element_by_xpath("/html/body/main/div/form/div/p/button").click()
    sleep(4)
    try:
      results = int(wd.find_element_by_xpath("/html/body/main/div/div/div/header/p[2]/span[1]").text.split(" ")[0])
    except:
      res = wd.find_element_by_xpath("/html/body/main/div/div/div/header/p[2]/span[1]").text.split(" ")[0]
      res = res.replace(",","")
      results = int(res)
    sleep(1)

    while True:
        sleep(2)
        x=0
        link = [j.get_attribute("href") for j in wd.find_elements_by_css_selector("a")]
        joblink = []
        for m in link:
            if "jobad" in m:
                joblink.append(m)
        print(len(joblink))
        if len(joblink) >= results:
            break
        while x<=500:
            try:
                wd.find_element_by_id("more-jobs").click()
                break
            except:
                scrolldown(1)
                x=x+1
    return(joblink)

def ScrapeVec(urlvec):
    title = []
    company = []
    for p in urlvec:
        soup = ScrapePage(p)
        try:
            title.append(soup.find_all("h1")[0].text)
        except:
            title.append("No title found")
        try:
            company.append(soup.findAll("p", {"class": "company"})[0].text.strip())
        except:
            company.append("No company found")
    ScrapedData = pd.DataFrame({
        "title": title,
        "company": company
    })
    return(ScrapedData)

def Main():
    role = ""
    location = "malta"
    urlvec = SearchScrape(role,location)
    jobdata = ScrapeVec(urlvec)
    jobdata["url"] = urlvec
    jobdata["date_added"] = str(datetime.date.today())
    jobdata.to_csv("ScrapedData.csv",index=False)
    CreateAppendDB(jobdata)
    print(jobdata)

if __name__ == '__main__':Main()

# Querying the database

Sending an email with new jobs

In [ ]:
db=sqlite3.connect("MaltaJobs.db")
new_result = pd.read_sql("select * from Jobs where date_added ='{}'".format(str(datetime.date.today())),db)
new_result.to_csv("new_result.csv",index=False)
new_list = "Kindly find new jobs below: \n"
for i in range(0,len(new_result)):
  string_new = new_result["title"][i] + " at " +  new_result["company"][i] + " added on " +  new_result["date_added"][i] + " with URL " + new_result["url"][i] + "\n"
  new_list += string_new
new_list = new_list.replace("\u2013","")
print(new_list)

import email, smtplib, ssl
from email import encoders
from email.mime.base import MIMEBase
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

email_list = ["lukejohnsaid1989@gmail.com"]

for i in email_list:
  subject = "Automated: New jobs today on Careerjet"
  body = "Good day! \n Attached kindly find new job listings which have been added today from Careerjet. "
  sender_email = 'lukejohnsaid1989@gmail.com'
  receiver_email = i
  password = '*****'

  # Create a multipart message and set headers
  message = MIMEMultipart()
  message["From"] = sender_email
  message["To"] = receiver_email
  message["Subject"] = subject
  message["Bcc"] = receiver_email  # Recommended for mass emails

  # Add body to email
  message.attach(MIMEText(body, "plain"))

  filename = "new_result.csv"  # In same directory as script

  # Open PDF file in binary mode
  with open(filename, "rb") as attachment:
      # Add file as application/octet-stream
      # Email client can usually download this automatically as attachment
      part = MIMEBase("application", "octet-stream")
      part.set_payload(attachment.read())

  # Encode file in ASCII characters to send by email    
  encoders.encode_base64(part)

  # Add header as key/value pair to attachment part
  part.add_header(
      "Content-Disposition",
      f"attachment; filename= {filename}",
  )

  # Add attachment to message and convert message to string
  message.attach(part)
  text = message.as_string()

  # Log in to server using secure context and send email
  context = ssl.create_default_context()
  with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
      server.login(sender_email, password)
      server.sendmail(sender_email, receiver_email, text)

Kindly find new jobs below: 
Internal Accountant at Quad Consultancy added on 2020-12-10 with URL https://www.careerjet.com.mt/jobad/mt4c0fed48ac7459e35d5e72890161cbdc
German speaking Night Shift only - Customer Support agent to world known client in Malta at Nordic Jobs Worldwide added on 2020-12-10 with URL https://www.careerjet.com.mt/jobad/mtee075877a26e971035966184ea2a1fd4
Backend Developer (PHP) at Videoslots added on 2020-12-10 with URL https://www.careerjet.com.mt/jobad/mte8b944b2fd54fc33e3de56d94c158825
Customer Service Agent (German speaking) at Videoslots added on 2020-12-10 with URL https://www.careerjet.com.mt/jobad/mtcf7ab5f7888d55ea52a5b6b415ef6e99
German-speaking Customer Support Agent at Aspire Global added on 2020-12-10 with URL https://www.careerjet.com.mt/jobad/mt6838306067013bbb1f519b45478cede1
Auditor at GRS Recruitment added on 2020-12-10 with URL https://www.careerjet.com.mt/jobad/mt828bbe8f24a7a64eee67e33243d28b9f
Quality Control  Laboratory Assistant at ASG Ph